# Temperature-dependent von Bertalanffy growth (VBG) model

This notebook fits temperature-dependent VBG models and adds diagnostic and sensitivity plots.


In [ ]:
# --- Data loading (robust) ---
# Option B: resolve data path relative to the *scripts* directory (this notebook's directory).
from pathlib import Path
import pandas as pd

scripts_dir = Path.cwd()
# If executed from repo root, the notebook path is scripts/...; otherwise we're already in scripts.
if (scripts_dir / 'scripts').is_dir():
    scripts_dir = scripts_dir / 'scripts'

data_path = scripts_dir / 'data' / 'temperature_vbg_data.csv'
if not data_path.exists():
    # Fallback: handle cases where notebook is run from within scripts but data is elsewhere
    alt = Path.cwd().parent / 'data' / 'temperature_vbg_data.csv'
    if alt.exists():
        data_path = alt

df = pd.read_csv(data_path)
display(df.head())


## Model fitting

Keep the existing model code cells in place. The plotting cells below assume you end up with:
- observed mass column name: `mass` (or `M`)
- age column name: `age`
- temperature column name: `temp` (in °C)
- predictions available for two models (S and D) either as arrays/series or via functions.

If your column names differ, adjust the small mapping in the plotting cell.


In [ ]:
# --- Existing model code cells remain above this point ---
# (No changes requested here.)


# Plots (diagnostics & sensitivity)

The following cells are appended *after* model fitting. They try to be conservative and will
work with a few common variable naming conventions.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')

# ---- Column name mapping (adjust here if needed) ----
def _first_existing(cols):
    for c in cols:
        if c in df.columns:
            return c
    return None

age_col  = _first_existing(['age', 'Age', 't', 'time'])
temp_col = _first_existing(['temp', 'Temp', 'temperature', 'Temperature', 'T'])
mass_col = _first_existing(['mass', 'Mass', 'M', 'weight', 'Weight'])

if age_col is None or temp_col is None or mass_col is None:
    raise ValueError(f'Could not infer required columns. Found: {list(df.columns)}')

age = np.asarray(df[age_col], dtype=float)
temp = np.asarray(df[temp_col], dtype=float)
obs  = np.asarray(df[mass_col], dtype=float)

# ---- Find predictions for Model S and D ----
# Supported patterns (any one is enough):
# 1) Arrays/Series named one of: pred_s, yhat_s, y_pred_s, pred_S
# 2) DataFrame columns: 'pred_s'/'pred_d' (etc.)
# 3) Callable functions: predict_s(age, temp), predict_d(age, temp)

def _get_pred(name_candidates, fn_candidates):
    # arrays in locals
    for n in name_candidates:
        if n in globals():
            v = globals()[n]
            try:
                return np.asarray(v, dtype=float)
            except Exception:
                pass
    # columns
    for n in name_candidates:
        if n in df.columns:
            return np.asarray(df[n], dtype=float)
    # functions
    for n in fn_candidates:
        if n in globals() and callable(globals()[n]):
            return np.asarray(globals()[n](age, temp), dtype=float)
    return None

pred_s = _get_pred(
    ['pred_s', 'pred_S', 'yhat_s', 'y_hat_s', 'y_pred_s', 'pred_model_s', 'y_pred_S'],
    ['predict_s', 'predict_S', 'model_s_predict', 'predict_model_s']
)
pred_d = _get_pred(
    ['pred_d', 'pred_D', 'yhat_d', 'y_hat_d', 'y_pred_d', 'pred_model_d', 'y_pred_D'],
    ['predict_d', 'predict_D', 'model_d_predict', 'predict_model_d']
)

if pred_s is None or pred_d is None:
    raise ValueError(
        'Could not locate predictions for both Model S and Model D. '
        'Expose arrays (pred_s/pred_d) or functions (predict_s/predict_d), or add df columns.'
    )

resid_s = obs - pred_s
resid_d = obs - pred_d

def rmse(r):
    r = np.asarray(r)
    return float(np.sqrt(np.mean(r**2)))

print('RMSE Model S:', rmse(resid_s))
print('RMSE Model D:', rmse(resid_d))


In [ ]:
# Observed vs Predicted (Model S and D)
fig, ax = plt.subplots(1, 2, figsize=(12, 5), sharex=True, sharey=True)
lims = [np.nanmin(obs), np.nanmax(obs)]
pad = 0.05 * (lims[1] - lims[0] if lims[1] > lims[0] else 1.0)
lo, hi = lims[0] - pad, lims[1] + pad

ax[0].scatter(obs, pred_s, s=18, alpha=0.7)
ax[0].plot([lo, hi], [lo, hi], 'k--', lw=1)
ax[0].set_title('Observed vs Predicted (Model S)')
ax[0].set_xlabel('Observed')
ax[0].set_ylabel('Predicted')
ax[0].set_xlim(lo, hi)
ax[0].set_ylim(lo, hi)

ax[1].scatter(obs, pred_d, s=18, alpha=0.7, color='tab:orange')
ax[1].plot([lo, hi], [lo, hi], 'k--', lw=1)
ax[1].set_title('Observed vs Predicted (Model D)')
ax[1].set_xlabel('Observed')
ax[1].set_ylabel('Predicted')
ax[1].set_xlim(lo, hi)
ax[1].set_ylim(lo, hi)

plt.tight_layout()
plt.show()


In [ ]:
# Residuals vs Age and Temperature
fig, ax = plt.subplots(2, 2, figsize=(12, 8), sharey='row')

ax[0, 0].scatter(age, resid_s, s=18, alpha=0.7)
ax[0, 0].axhline(0, color='k', lw=1)
ax[0, 0].set_title('Residuals vs Age (Model S)')
ax[0, 0].set_xlabel('Age')
ax[0, 0].set_ylabel('Observed - Predicted')

ax[0, 1].scatter(temp, resid_s, s=18, alpha=0.7)
ax[0, 1].axhline(0, color='k', lw=1)
ax[0, 1].set_title('Residuals vs Temperature (Model S)')
ax[0, 1].set_xlabel('Temperature (°C)')

ax[1, 0].scatter(age, resid_d, s=18, alpha=0.7, color='tab:orange')
ax[1, 0].axhline(0, color='k', lw=1)
ax[1, 0].set_title('Residuals vs Age (Model D)')
ax[1, 0].set_xlabel('Age')
ax[1, 0].set_ylabel('Observed - Predicted')

ax[1, 1].scatter(temp, resid_d, s=18, alpha=0.7, color='tab:orange')
ax[1, 1].axhline(0, color='k', lw=1)
ax[1, 1].set_title('Residuals vs Temperature (Model D)')
ax[1, 1].set_xlabel('Temperature (°C)')
plt.tight_layout()
plt.show()


In [ ]:
# Growth curves at multiple constant temperatures
temps = np.array(sorted(np.unique(temp)))
if len(temps) > 6:
    # choose representative temperatures
    temps = np.quantile(temps, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0])

t_grid = np.linspace(np.nanmin(age), np.nanmax(age), 200)

def _predict_grid(model, tvals, T):
    # model: 's' or 'd'
    Tvec = np.full_like(tvals, float(T), dtype=float)
    if model == 's':
        return np.asarray(_get_pred(['pred_s'], ['predict_s', 'predict_S'])(tvals, Tvec)
                          if callable(globals().get('predict_s', None)) or callable(globals().get('predict_S', None))
                          else np.interp(tvals, age, pred_s), dtype=float)
    else:
        return np.asarray(_get_pred(['pred_d'], ['predict_d', 'predict_D'])(tvals, Tvec)
                          if callable(globals().get('predict_d', None)) or callable(globals().get('predict_D', None))
                          else np.interp(tvals, age, pred_d), dtype=float)

# If no predictor functions exist, we can only show interpolated curves at observed temps/ages.
has_fn_s = any(n in globals() and callable(globals()[n]) for n in ['predict_s', 'predict_S', 'model_s_predict', 'predict_model_s'])
has_fn_d = any(n in globals() and callable(globals()[n]) for n in ['predict_d', 'predict_D', 'model_d_predict', 'predict_model_d'])

fig, ax = plt.subplots(1, 2, figsize=(13, 5), sharey=True)

for T in temps:
    if has_fn_s:
        y = np.asarray(globals().get('predict_s', globals().get('predict_S'))(t_grid, np.full_like(t_grid, T)))
        ax[0].plot(t_grid, y, label=f'{T:.1f}°C')
for T in temps:
    if has_fn_d:
        y = np.asarray(globals().get('predict_d', globals().get('predict_D'))(t_grid, np.full_like(t_grid, T)))
        ax[1].plot(t_grid, y, label=f'{T:.1f}°C')

ax[0].set_title('Growth curves (Model S)')
ax[1].set_title('Growth curves (Model D)')
for a in ax:
    a.set_xlabel('Age')
ax[0].set_ylabel('Predicted mass')
ax[0].legend(title='Temp', fontsize=9)

if not has_fn_s:
    ax[0].text(0.5, 0.5, 'No predict_s() found
Provide predictor function to draw curves',
              transform=ax[0].transAxes, ha='center', va='center')
if not has_fn_d:
    ax[1].text(0.5, 0.5, 'No predict_d() found
Provide predictor function to draw curves',
              transform=ax[1].transAxes, ha='center', va='center')

plt.tight_layout()
plt.show()


In [ ]:
# Asymptotic mass vs temperature
# If your model exposes an explicit asymptotic mass function/parameter, adapt here.
# Otherwise, approximate asymptote by predicted mass at max age on a grid.
temps_grid = np.linspace(np.nanmin(temp), np.nanmax(temp), 60)
t_max = float(np.nanmax(age))

has_fn_s = any(n in globals() and callable(globals()[n]) for n in ['predict_s', 'predict_S'])
has_fn_d = any(n in globals() and callable(globals()[n]) for n in ['predict_d', 'predict_D'])

asym_s = None
asym_d = None
if has_fn_s:
    f = globals().get('predict_s', globals().get('predict_S'))
    asym_s = np.array([f(np.array([t_max]), np.array([T]))[0] for T in temps_grid], dtype=float)
if has_fn_d:
    f = globals().get('predict_d', globals().get('predict_D'))
    asym_d = np.array([f(np.array([t_max]), np.array([T]))[0] for T in temps_grid], dtype=float)

fig, ax = plt.subplots(figsize=(8, 4.8))
if asym_s is not None:
    ax.plot(temps_grid, asym_s, label='Model S')
if asym_d is not None:
    ax.plot(temps_grid, asym_d, label='Model D')
ax.set_title('Approx. asymptotic mass vs temperature (at max age)')
ax.set_xlabel('Temperature (°C)')
ax.set_ylabel('Predicted mass at max age')
ax.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Q10 sensitivity plots
# Q10(T) for predicted mass growth between T and T+10 at a fixed age.
# Define Q10 = rate(T+10)/rate(T). Here we use predicted mass at age t_ref as proxy.
# If you have a direct temperature-dependent rate parameter, replace accordingly.

t_ref = float(np.nanmedian(age))
Tmin, Tmax = float(np.nanmin(temp)), float(np.nanmax(temp))
Tq = np.linspace(Tmin, Tmax - 10.0, 50)

def q10_from_predict(predict_fn):
    y1 = np.array([predict_fn(np.array([t_ref]), np.array([T]))[0] for T in Tq], dtype=float)
    y2 = np.array([predict_fn(np.array([t_ref]), np.array([T+10.0]))[0] for T in Tq], dtype=float)
    # avoid division by zero
    return y2 / np.where(np.abs(y1) < 1e-12, np.nan, y1)

fig, ax = plt.subplots(figsize=(8, 4.8))
if any(n in globals() and callable(globals()[n]) for n in ['predict_s', 'predict_S']):
    f = globals().get('predict_s', globals().get('predict_S'))
    ax.plot(Tq, q10_from_predict(f), label='Model S')
if any(n in globals() and callable(globals()[n]) for n in ['predict_d', 'predict_D']):
    f = globals().get('predict_d', globals().get('predict_D'))
    ax.plot(Tq, q10_from_predict(f), label='Model D')

ax.axhline(2.0, color='k', lw=1, ls='--', alpha=0.7)
ax.set_title(f'Q10 sensitivity (proxy) at age t={t_ref:.2f}')
ax.set_xlabel('Temperature T (°C), compares T to T+10')
ax.set_ylabel('Q10 (proxy)')
ax.set_ylim(bottom=0)
ax.legend()
plt.tight_layout()
plt.show()
